In [1]:
import numpy as np
import math
import scripts as sc
O = 0.7
C = 0.78
H = 0.35
N = 0.71
P = 1.08

**Read the file and save all data**

In [2]:
with open('Task3.pdb', 'r') as file:
    lines = file.readlines()

**FIND EXACT PART IN FILE(our variant)**

In [3]:
task3_array = []
index = 0

while True:
    if 'MO' in lines[index]:
        if "13" in lines[index]:
            index += 3
            break
    index += 82

**Заповнюємо наш головний масив саме нашим 13 варіантом**

In [4]:
while lines[index][0] != 'C':
    task3_array.append(lines[index].split())
    index += 1

**Created a separate array with the names of DNA skeletal atoms and their coordinates**

In [5]:
special_atom_names_with_coordinates = [0]*9
needed_atoms = ["O3'1", "P2", "O5'2", "C5'2", "C4'2", "C3'2", "C5'1", "C4'1", "C3'1"]

**Read the file and identify every needed atom**

In [6]:
for index in range(len(task3_array)):
    if task3_array[index][2][0:2] == "C5" and task3_array[index][4] == "1":
        special_atom_names_with_coordinates[6] = [task3_array[index][2]+"1", task3_array[index][5], task3_array[index][6], task3_array[index][7]]
    elif task3_array[index][2][0:2] == "C4" and task3_array[index][4] == "1":
        special_atom_names_with_coordinates[7] = [task3_array[index][2]+"1", task3_array[index][5], task3_array[index][6], task3_array[index][7]]
    elif task3_array[index][2][0:2] == "C3" and task3_array[index][4] == "1":
        special_atom_names_with_coordinates[8] = [task3_array[index][2]+"1", task3_array[index][5], task3_array[index][6], task3_array[index][7]]
    elif task3_array[index][2][0:2] == "O3" and task3_array[index][4] == "1":
        special_atom_names_with_coordinates[0] = [task3_array[index][2]+"1", task3_array[index][5], task3_array[index][6], task3_array[index][7]]
    elif task3_array[index][2] == "P" and task3_array[index][4] == "2":
        special_atom_names_with_coordinates[1] = [task3_array[index][2]+"2", task3_array[index][5], task3_array[index][6], task3_array[index][7]]
    elif task3_array[index][2] == "O5'" and task3_array[index][4] == "2":
        special_atom_names_with_coordinates[2] = [task3_array[index][2]+"2", task3_array[index][5], task3_array[index][6], task3_array[index][7]]
    elif task3_array[index][2] == "C5'" and task3_array[index][4] == "2":
        special_atom_names_with_coordinates[3] = [task3_array[index][2]+"2", task3_array[index][5], task3_array[index][6], task3_array[index][7]]
    elif task3_array[index][2] == "C4'" and task3_array[index][4] == "2":
        special_atom_names_with_coordinates[4] = [task3_array[index][2]+"2", task3_array[index][5], task3_array[index][6], task3_array[index][7]]
    elif task3_array[index][2] == "C3'" and task3_array[index][4] == "2":
        special_atom_names_with_coordinates[5] = [task3_array[index][2]+"2", task3_array[index][5], task3_array[index][6], task3_array[index][7]]

**Write needed dihedral angles**

In [7]:
alpha = ["O3'1", "P2", "O5'2", "C5'2"]
beta = ["P2", "O5'2", "C5'2", "C4'2"]
gamma = ["O5'2", "C5'2", "C4'2", "C3'2"]
delta = ["C5'1", "C4'1", "C3'1", "O3'1"]
epsilon = ["C4'1", "C3'1", "O3'1", "P2"]
zeta = ["C3'1", "O3'1", "P2", "O5'2"]
angles = [alpha, beta, gamma, delta, epsilon, zeta]
angles_names = ["alpha", "beta", "gamma", "delta", "epsilon", "zeta"]
valent_angles = []

**Find valent angles**

In [8]:
for i in range(0, 6):
    valent_angles.append(sc.calculate_angle(angles[i], special_atom_names_with_coordinates, needed_atoms))

**Find dihedral angles**

In [12]:
ps = []
tors_angles = []
for i in range(0, 6):
    ps.append(sc.Vector(angles[i], needed_atoms, special_atom_names_with_coordinates).result)
for i in range(0,6):
    tors_angles.append(sc.angle(sc.vector_cross(ps[i][0], ps[i][1]), sc.vector_cross(ps[i][1], ps[i][2])))
    tors_angles[i] *= sc.adjusment(sc.vector_cross(ps[i][0], ps[i][1]), sc.vector_cross(ps[i][1], ps[i][2]), ps[i][1])

**Create alpha and phi matrixies for later manipulations**

In [14]:
alpha_matrixies = []
for i in range(0, 6):
    temp_matrix = np.array([
        [-math.cos(valent_angles[i]*math.pi/180), -math.sin(valent_angles[i]*math.pi/180), 0],
        [math.sin(valent_angles[i]*math.pi/180), -math.cos(valent_angles[i]*math.pi/180), 0],
        [0, 0, 1]
    ])

    alpha_matrixies.append(temp_matrix)




phi_matrixies = []
for i in range(0, 6):
    temp_matrix = np.array([
    [1, 0, 0],
    [0, math.cos(tors_angles[i]*math.pi/180), -math.sin(tors_angles[i]*math.pi/180)],
    [0, math.sin(tors_angles[i]*math.pi/180), math.cos(tors_angles[i]*math.pi/180)]
    ])
    phi_matrixies.append(temp_matrix)

**Multiply them**

In [15]:
A_matrix = alpha_matrixies[0]
for i in range(0, 5):
    A_matrix = np.dot(A_matrix, phi_matrixies[i])
    A_matrix = np.dot(A_matrix, alpha_matrixies[i+1])
A_matrix = np.dot(A_matrix, phi_matrixies[5])

**And now we find our angle "theta"**

In [20]:
cos_theta_2 = (math.sqrt(1+A_matrix[0][0]+A_matrix[1][1] + A_matrix[2][2]))/2
theta = np.arccos(cos_theta_2)*2
print("theta = ", theta*180/math.pi)

theta =  178.93893078094325


In [23]:
with open("results_of_laba3", "w") as file:
    file.write("theta = " + str(theta*180/math.pi) + "\n")